# How-to: Predict and update URLs for OPD data

#### TO-DO
- complete testing
- Might need requirements.txt and how to code for that, using % to install packages
- add check if URL already in source table
- Is there a get data loader function?

## Env and packages

In [19]:
import prediction_funcs

## Example: Try to find a valid URL for a new year

In [ ]:
url = "https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2016_csv/FeatureServer/0"
spreadsheet_fields = {
    "State": "CA",
    "SourceName": "Example PD",
    "Agency": "EXPD",
    "AgencyFull": "Example Police Department",
    "TableType": "Arrests",
    "DataType": "ArcGIS"
}

prediction_funcs.try_url_years(url, n_years=3, forward=True, extra_fields=extra_fields)

Valid URL found: https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2017_csv/FeatureServer/0 (status: ArcGIS check)
Added to OPD_Source_table: https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2017_csv/FeatureServer/0


'https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2017_csv/FeatureServer/0'

In [ ]:
import requests

In [ ]:
resp = requests.head(url, allow_redirects=False, timeout=10)
resp.status_code

## Example: Automatically update all outdated sources

In [ ]:
prediction_funcs.auto_update_sources(verbose=True)